In [1]:
import pandas as pd
import numpy as np
import data_transformation as dt
import feature_engineering as fe
import pickle
from testing.predictiveAccuracy import metric, custom_scorer_metric
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split, TimeSeriesSplit, KFold, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn import linear_model

In [2]:
load = True

In [3]:
raw_data = dt.get_all_train_data()

/home/chris/anaconda3/envs/minicomp/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
sorted_data = raw_data.sort_values(by=['Date'])

In [5]:
target_col = 'Sales'

In [10]:
# make time series split for cross validation

kf = TimeSeriesSplit(n_splits=5)
X_train_folds = []
X_test_folds = []
y_train_folds = []
y_test_folds = []
for train_index, test_index in kf.split(sorted_data):
    
    train_data = raw_data.iloc[train_index, :]
    y_train_folds.append(train_data)
    X_train_folds.append(train_data)
    
    test_data = raw_data.iloc[test_index, :]
    y_test_folds.append(test_data)
    X_test_folds.append(test_data)

In [ ]:
# apply imputing to each set
X_train_folds_imp = []
X_test_folds_imp = []
y_train_folds_imp = []
y_test_folds_imp = []

for fold in X_train_folds:
    X_train_folds_imp.append(dt.create_basetable(fold))
for fold in X_test_folds:
    X_test_folds_imp.append(dt.create_basetable(fold))
for fold in y_train_folds:
    y_train_folds_imp.append(dt.create_basetable(fold))
for fold in y_test_folds:
    y_test_folds_imp.append(dt.create_basetable(fold))

In [6]:
split_clean_data_path = './data/chris_split_clean_data_list.pkl'

In [7]:
if not load:
    with open(split_clean_data_path, 'wb') as f:
        pickle.dump([X_train_folds_imp, X_test_folds_imp, y_train_folds_imp, y_test_folds_imp], f)

In [8]:
if load:
    saved_folds = pickle.load(open(split_clean_data_path, "rb" ))
    X_train_folds_imp = saved_folds[0]
    X_test_folds_imp = saved_folds[1]
    y_train_folds_imp = saved_folds[2]
    y_test_folds_imp = saved_folds[3]

In [11]:
# grid search on linear regression
model = linear_model.LinearRegression()
params = {
    'fit_intercept': [True, False],
    'normalize': [True, False]
}
grid = GridSearchCV(estimator=model, param_grid=params, scoring=make_scorer(custom_scorer_metric, greater_is_better=False), n_jobs=3, cv=kf.split(sorted_data), verbose=True)

In [ ]:
# fold sizes
for folds in [X_train_folds_imp, X_test_folds_imp, y_train_folds_imp, y_test_folds_imp]:
    for fold in folds:
        print(fold.shape)
        
print(len(X_train_folds_imp[-1]) + len(X_test_folds_imp[-1]))    
sorted_data.shape

In [12]:
# restack imputed folds for the gridsearch
restacked_X_data = X_train_folds_imp[-1].append(X_test_folds_imp[-1])
restacked_y_data = y_train_folds_imp[-1].append(y_test_folds_imp[-1])

# pd.get_dummies(restacked_X_data).dtypes

In [ ]:
grid.fit(
    restacked_X_data.drop(['Date', 'Sales'], axis=1).to_numpy(),
    restacked_y_data['Sales'].to_numpy()
)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


In [ ]:
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X_train_no_date, y_train)

In [ ]:
preds = regr.predict(X_test_no_date)

In [ ]:
# random forest prediction
metric(preds, y_test.to_numpy())

In [ ]:
# average prediction
metric(np.mean(X_test_no_date.to_numpy(), axis=1), y_test.to_numpy())